# Buoy data retrieval
The URL to get the data will roughly be something like this: https://api.riddc.brown.edu/telemetry/Buoy-###/CoreMetrics/range?start=YYYY-MM-DD&end=YYYY-MM-DD

Note how in the URL, there is a place for buoy number. Buoy 620 is at Jamestown. Buoy 720 is in Greenwich Bay. There is also a place to specify the start date and end date of the data you want.

So let's say you want data from Buoy 720 from January 1, 2023 to February 1, 2023. The URL will be https://api.riddc.brown.edu/telemetry/Buoy-720/CoreMetrics/range?start=2023-01-01&end=2023-02-01.

Once at the website, in the upper left hand corner, click JSON. This is the most readable form of the data. The header `MetData` gives data from the meteorological sensor. This has wind.

I wrote a function in Python that automates data retrieval from these sensors. 

In [1]:
def get_buoy(start_date, end_date, buoy, sensor):
    """
    Retrieve data from Andy Davies' sensors using the API
    Choose from PAR, SUNA, or MetData sensors for, respectively,
        - light
        - nutrients
        - meteorological data
    Drop 0 or negative pH values
    Time zone kept in UTC
        
    INPUTS:
        - start_date: date as a string, formatted as YYYY-MM-DD
        - end_date: date as a string, formatted as YYYY-MM-DD
         - buoy: Jamestown/620/PLT or 720/Greenwich Bay/BG
        - PAR, SUNA, or MetData sensors
        
    RETURNS pandas DataFrame
    """
    import re
    import time
    import requests
    import pandas as pd
    import numpy as np
    from datetime import datetime, timedelta
    import math
    
    # Check that start and end dates are formatted correctly
    # Return error message if format is wrong
    matched_start = re.match("\d{4}-\d{2}-\d{2}", start_date)
    matched_end = re.match("\d{4}-\d{2}-\d{2}", end_date)
    if bool(not matched_start) or bool(not matched_end):
        print('Error: Input date must be a string of the form YYYY-MM-DD')
        return
    
    # Check that start date is before end date
    # Switch dates if start date after end date
    a = time.strptime(start_date, '%Y-%m-%d')
    b = time.strptime(end_date, '%Y-%m-%d')
    if a > b:
        start = start_date
        start_date = end_date
        end_date = start
    
    # create URL string using start date, end date, and buoy
    if buoy == 'Jamestown' or buoy == '620' or buoy == 'PLT': 
        URL = 'https://api.riddc.brown.edu/telemetry/Buoy-620/CoreMetrics/range?start=' + start_date + '&end=' + end_date
    elif buoy == 'Greenwich Bay' or buoy == '720' or buoy == 'GB': 
        URL = 'https://api.riddc.brown.edu/telemetry/Buoy-720/CoreMetrics/range?start=' + start_date + '&end=' + end_date
    else:
        print('Buoy options are 620 for Jamestown/PLT or 720 for Greenwich Bay/GB.')
        return                                                                                                          
    
    # Ingest data from API
    r = requests.get(URL)
    JSON = r.json()
    df = pd.DataFrame(JSON[sensor])
    
    # Drop 0 and negative values and nan values
    for k in df.keys():
        for ind in df.index:
            if isinstance(df[k][ind], float) or isinstance(df[k][ind], int):
                if math.isnan(df[k][ind]):
                    df = df.drop(ind)
                    df = df.reset_index(drop=True)
            
    # Get DateTime object from TmStamp string 
    # Keep timezone in UTC
    df['DateTime'] = np.zeros(len(df['TmStamp']))
    for ind in df.index:
        df['DateTime'][ind] = datetime.strptime(df['TmStamp'][ind], '%Y-%m-%dT%H:%M:%S.%fZ') 
    
    return df

In [2]:
df = get_buoy('2023-01-01', '2023-02-01', '720', 'MetData')
df

/var/folders/9k/5r38tm8d21g6nm3w9rchrd6m0000gn/T/ipykernel_53422/255598899.py:72: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['DateTime'][ind] = datetime.strptime(df['TmStamp'][ind], '%Y-%m-%dT%H:%M:%S.%fZ')


,TmStamp,metdataStart,avgWindSpeed,avgWindDir,gustWindSpeed,gustWindDir,maximetTemperature,maximetPressure,maximetHumidity,maximetPrecipitation,maximetSolar,DateTime
0,2023-01-01T00:00:00.000Z,01/01/2023 00:00:00,3.00852,205.4970,3.89278,205.2080,6.7,1006.5,99,0.40,1,2023-01-01 00:00:00
1,2023-01-01T00:10:00.000Z,01/01/2023 00:10:00,3.37544,191.7340,4.09727,184.8460,6.6,1006.4,99,0.32,1,2023-01-01 00:10:00
2,2023-01-01T00:20:00.000Z,01/01/2023 00:20:00,3.86973,195.7620,5.09213,202.6750,7.3,1006.5,99,0.48,1,2023-01-01 00:20:00
3,2023-01-01T00:30:00.000Z,01/01/2023 00:30:00,4.52461,196.1550,5.13298,194.9380,7.0,1006.2,99,0.64,1,2023-01-01 00:30:00
4,2023-01-01T00:40:00.000Z,01/01/2023 00:40:00,4.14798,195.2640,4.83217,198.3040,6.8,1006.0,99,0.72,1,2023-01-01 00:40:00
...,...,...,...,...,...,...,...,...,...,...,...,...
4336,2023-01-31T23:10:00.000Z,01/31/2023 23:10:00,7.58466,44.0005,10.07470,50.7384,-1.1,1024.0,59,0.00,0,2023-01-31 23:10:00
4337,2023-01-31T23:20:00.000Z,01/31/2023 23:20:00,8.02195,41.5288,10.18840,56.7145,-1.1,1024.4,61,0.00,0,2023-01-31 23:20:00
4338,2023-01-31T23:30:00.000Z,01/31/2023 23:30:00,7.66375,39.4002,9.74613,31.9556,-1.3,1024.4,58,0.00,0,2023-01-31 23:30:00
4339,2023-01-31T23:40:00.000Z,01/31/2023 23:40:00,6.86726,35.4995,8.73523,45.4349,-1.4,1024.1,58,0.00,0,2023-01-31 23:40:00


You can save the resulting dataframe to a CSV locally as follows: 

```python
df.to_csv('/Users/akbaskind/Desktop/YoE/DOCUMENT.csv')
```